In [1]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

In [2]:
# Clone
repo_path = "/home/rupesh/aqrtr/ai/langchain/proj-101-go/go_repo-01"


In [ ]:
repo = Repo.clone_from("https://github.com/rupeshtr78/nvidia-server", to_path=repo_path)

In [4]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".go"],
    exclude=["Dockerfile", "vendor", "docker-compose.yml", "Makefile", "README.md"],
    parser=LanguageParser(language=Language.GO, parser_threshold=500),
)
documents = loader.load()
len(documents)

10

In [5]:
for doc in documents:
    print(doc.metadata)
    print("\n\n\n")

{'source': '/home/rupesh/aqrtr/ai/langchain/proj-101-go/go_repo-01/internal/metrics/fetch_gpu_metrics.go', 'language': <Language.GO: 'go'>}




{'source': '/home/rupesh/aqrtr/ai/langchain/proj-101-go/go_repo-01/internal/metrics/gpu_device.go', 'language': <Language.GO: 'go'>}




{'source': '/home/rupesh/aqrtr/ai/langchain/proj-101-go/go_repo-01/internal/metrics/gpu_metrics.go', 'language': <Language.GO: 'go'>}




{'source': '/home/rupesh/aqrtr/ai/langchain/proj-101-go/go_repo-01/internal/metrics/gpu_device_test.go', 'language': <Language.GO: 'go'>}




{'source': '/home/rupesh/aqrtr/ai/langchain/proj-101-go/go_repo-01/internal/metrics/fetch_gpu_mertics_test.go', 'language': <Language.GO: 'go'>}




{'source': '/home/rupesh/aqrtr/ai/langchain/proj-101-go/go_repo-01/internal/metrics/fetch_gpu_metrics_v2_test.go', 'language': <Language.GO: 'go'>}




{'source': '/home/rupesh/aqrtr/ai/langchain/proj-101-go/go_repo-01/internal/metrics/gpu_metrics_test.go', 'language': <Language.GO: 'go'>}

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

go_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.GO, chunk_size=2000, chunk_overlap=200
)
texts = go_splitter.split_documents(documents)
len(texts)

18

In [7]:
texts[0]

Document(page_content='package gpumetrics\n\nimport (\n\t"context"\n\t"fmt"\n\t"log"\n\t"sync"\n\n\t"github.com/NVIDIA/go-nvml/pkg/nvml"\n)\n\n// FetchGpuInfo fetches the metrics for all GPU devices\nfunc FetchAllGpuInfo(ctx context.Context, gpu GpuDeviceManager, count int) (GpuMap, error) {\n\n\tif count == 0 {\n\t\treturn nil, fmt.Errorf("no GPU devices found")\n\t}\n\n\tgpuMap := make(GpuMap)\n\n\t// gpuChan := make(chan GpuMap, count)\n\t// defer close(gpuChan)\n\n\terrChan := make(chan error, 1) // only need to store one error\n\tdefer close(errChan)\n\n\twg := new(sync.WaitGroup)\n\twg.Add(count)\n\n\tfor i := 0; i < count; i++ {\n\t\tgo func(i int) {\n\t\t\tdefer wg.Done()\n\n\t\t\t// context done means there has been a cancellation signal\n\t\t\tselect {\n\t\t\tcase <-ctx.Done():\n\t\t\t\terrChan <- ctx.Err()\n\t\t\t\treturn\n\t\t\tdefault:\n\t\t\t}\n\n\t\t\tdevice, ret := gpu.DeviceGetHandleByIndex(i)\n\t\t\tif ret != nvml.SUCCESS {\n\t\t\t\terrChan <- fmt.Errorf("failed to ge

In [8]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [8]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model="gpt-4")
memory = ConversationSummaryMemory(llm=llm, max_memory_length=5, max_memory_turns=5, return_messages=True, memory_key="chat_history")
qaChain = ConversationalRetrievalChain.from_llm(llm=llm, memory=memory, retriever=retriever)

In [10]:
# Ask a question
question = "Can you suggest some improvements for function FetchAllGpuInfo?"  # Replace with your actual question

# Include 'chat_history' key if required by your function
formatted_question = {'question': question, 'chat_history': ''}

# Invoke the conversation chain with the properly formatted question
result = qaChain.invoke(formatted_question)

# Fetch and print the answer
answer = result['answer']
print(answer)



Number of requested results 20 is greater than number of elements in index 18, updating n_results = 18


1. Error Handling: The `FetchAllGpuInfo` function could use more comprehensive error handling. Right now, it only returns the first error it encounters, but it could be more informative if it collected all errors and returned them together.

2. Parallelism Control: The function spawns a goroutine for each GPU device, which could potentially lead to issues if the count of GPU devices is large. It could be improved by using a worker pool pattern to limit the number of concurrent goroutines.

3. Context Usage: The function could make better use of the context passed in. It could pass the context to the `FetchDeviceMetrics` function to allow for cancellation of that function as well.

4. Logging: The function could use structured logging to make the logs easier to parse and analyze. Additionally, it could include more information in the logs, such as the specific GPU device that an operation is being performed on.

5. Code Duplication: The function has some duplicated code that could be re

In [13]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_models.ollama import ChatOllama
llmOllama = ChatOllama(model="llama3:70b")
memoryOllama = ConversationSummaryMemory(llm=llmOllama, max_memory_length=5, max_memory_turns=5, return_messages=True, memory_key="chat_history")
ollamaChain = ConversationalRetrievalChain.from_llm(llm=llmOllama, memory=memoryOllama, retriever=retriever)

In [14]:
# Ask a question
question = "Can you write tests for function FetchAllGpuInfo?"  # Replace with your actual question

# Include 'chat_history' key if required by your function
formatted_question = {'question': question, 'chat_history': ''}

# Invoke the conversation chain with the properly formatted question
result = ollamaChain.invoke(formatted_question)

# Fetch and print the answer
answer = result['answer']
print(answer)

Number of requested results 20 is greater than number of elements in index 18, updating n_results = 18


Writing tests for the `FetchAllGpuInfo` function involves creating a test scenario where you can control the inputs and expected outputs. Here's an example of how you can approach this:

**Test Strategy**

1. Create a mock implementation of the `GpuDeviceManager` interface, which will allow you to control the behavior of the `DeviceGetHandleByIndex`, `GetUUID`, `GetName`, `GetTemperature`, `GetPowerUsage`, and `GetMemoryInfo` methods.
2. Write test cases for different scenarios:
	* Happy path: all GPU info is fetched successfully
	* Error handling: one or more GPU info fetching fails
	* Edge cases: empty list of GPUs, invalid GPU indices, etc.

**Example Test Code**

Here's an example of how you can write tests for the `FetchAllGpuInfo` function:
```go
package gpumetrics

import (
	"context"
	"testing"

	"github.com/stretchr/testify/assert"
)

func TestFetchAllGpuInfo(t *testing.T) {
	mockDeviceManager := new(MockGpuDeviceManager)

	// Happy path test
	t.Run("HappyPath", func(t *testin

In [15]:
# Ask a question
question = "Can you write MockGpuDeviceManager?"  # Replace with your actual question

# Include 'chat_history' key if required by your function
formatted_question = {'question': question, 'chat_history': ''}

# Invoke the conversation chain with the properly formatted question
result = ollamaChain.invoke(formatted_question)

# Fetch and print the answer
answer = result['answer']
print(answer)

Number of requested results 20 is greater than number of elements in index 18, updating n_results = 18


To implement a `MockGpuDeviceManager` for testing purposes, you can create a struct that satisfies the `GpuDeviceManager` interface and provides mock implementations for its methods.

Here's an example implementation:
```go
type MockGpuDeviceManager struct {
    // Mocked method return values
    DeviceGetCountReturn  int
    DeviceGetCountErr     nvml.Return
    DeviceGetHandleByIndexReturn   *nvml.Device
    DeviceGetHandleByIndexErr      nvml.Return
    // ... add more mocked method return values as needed

    // Call counters for testing purposes
    DeviceGetCountCallCount  int
    DeviceGetHandleByIndexCallCount int
    // ... add more call counters as needed
}

func (m *MockGpuDeviceManager) DeviceGetCount() (int, nvml.Return) {
    m.DeviceGetCountCallCount++
    return m.DeviceGetCountReturn, m.DeviceGetCountErr
}

func (m *MockGpuDeviceManager) DeviceGetHandleByIndex(index int) (*nvml.Device, nvml.Return) {
    m.DeviceGetHandleByIndexCallCount++
    return m.DeviceGetHandle

In [16]:
# Ask a question
question = "Can you write MockNvmlDevice?"  # Replace with your actual question

# Include 'chat_history' key if required by your function
formatted_question = {'question': question, 'chat_history': ''}

# Invoke the conversation chain with the properly formatted question
result = ollamaChain.invoke(formatted_question)

# Fetch and print the answer
answer = result['answer']
print(answer)

Number of requested results 20 is greater than number of elements in index 18, updating n_results = 18


To implement a `MockNvmlDevice` in Go, you can use the `mock` package from the `github.com/stretchr/testify/mock` library. This package provides a way to create mock objects that satisfy interfaces.

Here's an example implementation of a `MockNvmlDevice`:
```go
import (
	"github.com/stretchr/testify/mock"
	"gitlab.com/nvidia/go-nvml/pkg/nvml"
)

type MockNvmlDevice struct {
	mock.Mock
}

func (m *MockNvmlDevice) DeviceGetCount() (int, nvml.Return) {
	args := m.Called()
	return args.Get(0).(int), args.Get(1).(nvml.Return)
}

func (m *MockNvmlDevice) DeviceGetHandleByIndex(index int) (*nvml.Device, nvml.Return) {
	args := m.Called(index)
	return args.Get(0).(*nvml.Device), args.Get(1).(nvml.Return)
}

func (m *MockNvmlDevice) FetchDeviceMetrics(device *nvml.Device) (*NvidiaDevice, nvml.Return) {
	args := m.Called(device)
	return args.Get(0).(*NvidiaDevice), args.Get(1).(nvml.Return)
}
```
In this implementation, we define a `MockNvmlDevice` struct that embeds the `mock.Mock` type from th